In [35]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [36]:
list_folder = os.listdir('train')

raw_x = []
raw_y_traget = []
for i in list_folder:
    list_file_wave = os.listdir('train' + '/' + i)
    for j in list_file_wave:
        filename = 'train' + '/' + i + '/' + j
        data, sr = librosa.load(filename, sr = 44100)
        
        raw_x.append(data)
        raw_y_traget.append(i)

df = []
dfs = []
for y in raw_x:
    melspectrogram = librosa.feature.melspectrogram(y=y, sr=22050, n_mels=128,fmax = 8000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc = 40)


    for i in range (melspectrogram.shape[0]):
        df.append(melspectrogram[i].mean())
        df.append(melspectrogram[i].std())
        df.append(melspectrogram[i].var())
   
    for i in range (mfcc.shape[0]):
        df.append(mfcc[i].mean())
        df.append(mfcc[i].std())
        df.append(mfcc[i].var())
        
    dfs.append(df)
    df = []

In [37]:
raw_x = []
raw_y = []
list_file_wave = os.listdir('test')
for j in list_file_wave:
    filename = 'test' + '/' + j
    data, sr = librosa.load(filename, sr = 44100)
    
    raw_x.append(data)
    raw_y.append(j)
  
df = []
dfys = []
for y in raw_x:
    melspectrogram = librosa.feature.melspectrogram(y=y, sr=22050, n_mels=128,fmax = 8000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc = 40)

    for i in range (melspectrogram.shape[0]):
        df.append(melspectrogram[i].mean())
        df.append(melspectrogram[i].std())
        df.append(melspectrogram[i].var())
   
    for i in range (mfcc.shape[0]):
        df.append(mfcc[i].mean())
        df.append(mfcc[i].std())
        df.append(mfcc[i].var())
                
    dfys.append(df)
    df = []

In [38]:
df_train = pd.DataFrame(dfs)
df_test = pd.DataFrame(dfys)

In [39]:
y_traget = pd.DataFrame(raw_y_traget, columns=['result'])

In [40]:
scaler = StandardScaler()
df_train_scaler = scaler.fit_transform(df_train)
df_test_scaler = scaler.transform(df_test)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(df_train_scaler, y_traget, test_size=0.1,random_state=42)

In [42]:
param_grid = { 
    'n_estimators': [200, 1000, 2000, 4000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [None, 10, 50, 100],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4],
    'criterion' :['gini', 'entropy']}

In [ ]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, refit=True, verbose=3)
CV_rfc.fit(df_train_scaler, y_traget)

In [ ]:
CV_rfc.best_params_

In [43]:
rfc1=RandomForestClassifier(random_state=42, max_features='log2', n_estimators= 4000, max_depth=None,
                            min_samples_leaf= 1, min_samples_split=2, bootstrap=False,criterion='entropy')

In [44]:
rfc1.fit(X_train, y_train)
rfc1.score(X_test,y_test)

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.9

In [95]:
rfc1.fit(df_train_scaler, y_traget)

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(bootstrap=False, criterion='entropy',
                       max_features='log2', n_estimators=4000, random_state=42)

In [96]:
y_pred = rfc1.predict(df_test_scaler)

In [97]:
submiss = pd.read_csv("sample_submission.csv")

In [98]:
submiss = submiss.sort_values(['files'])

In [99]:
submiss['label'] = y_pred

In [100]:
submiss = submiss.sort_index()

In [22]:
submiss.to_csv('RandomForestClassifier12.csv', index=False)